In [ ]:
pip install colorama

In [ ]:
pip install tensorflowjs

In [3]:
import zipfile #módulo para manipulação de arquivos zipados
import pandas as pd
from google.colab import drive
import cv2
import numpy as np
import requests as req
from PIL import Image
from io import BytesIO
from google.colab.patches import cv2_imshow
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet import ResNet152
import os, cv2, numpy as np
from matplotlib import pyplot as plt #importa bilbioteca para criação de gráficos
from sklearn.model_selection import train_test_split
from os import listdir
from keras.utils.np_utils import to_categorical
from keras.utils.layer_utils import count_params
from keras.models import Sequential
from keras.layers import InputLayer, BatchNormalization
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
import tensorflow as tf

from colorama import init, Fore, Back, Style
import tensorflowjs as tfjs

In [4]:
# Monta Google Drive
drive.mount('/content/drive')
# Caminho para o arquivo baixado
local_zip = '/content/drive/My Drive/AI/IGTI/Bootcamp Deep Learning/Desafio Final/dataset.zip'
# Abre arquivo zipado
zip_ref = zipfile.ZipFile(local_zip, 'r')
# Extrai arquivo zipado para diretório /tmp
zip_ref.extractall('/tmp')
# Fecha arquivo baixado
zip_ref.close()

Mounted at /content/drive


In [19]:
base_dir = '/tmp/dataset'
mask_dir = os.path.join(base_dir, 'with_mask') # Diretório com imagens de treinamentos de gatos
no_mask_dir = os.path.join(base_dir, 'without_mask') # Diretório com imagens de treinamentos de cachorros
HEIGHT = 64
WIDTH = 64
data = {
    'filename': [],
    'img': [],
    'class': []
}
_epochs = 50

In [20]:
def reset_data (h = 64, w = 64):
  global HEIGHT
  global WIDTH
  global data
  
  data = {
      'filename': [],
      'img': [],
      'class': []
  }
  HEIGHT = h
  WIDTH = w

In [21]:
# Le um diretorio e joga pro array de dados
def read_folder (folder, with_mask):

  for fname in listdir(folder):
    fpath = os.path.join(folder, fname)
    image = cv2.imread(fpath)
    image_resized = cv2.resize(image, (HEIGHT, WIDTH))
    data['filename'].append(fpath) # caminho do arquivo
    data['img'].append(image_resized) # a imagem vai pro X
    data['class'].append(with_mask) # a label vai pro Y

In [22]:
def get_model(imgs_shape = (HEIGHT, WIDTH, 3), act = 'relu'):
  model = Sequential()

  print(imgs_shape)

  # camada de entrada
  #model.add(InputLayer(input_shape=imgs_shape))

  # convolucional 1
  model.add(Conv2D(64, (7, 7), activation=act, input_shape=imgs_shape))
  # pooling 1
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # convolucional 2
  model.add(Conv2D(128, (5, 5), activation=act))
  # pooling 2
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # convolucional 3
  model.add(Conv2D(256, (5, 5), activation=act))
  # pooling 3
  model.add(MaxPooling2D(pool_size=(2, 2)))

  # convolucional 4
  model.add(Conv2D(512, (3, 3), activation=act))

  # convolucional 5
  model.add(Conv2D(1024, (3, 3), activation=act, padding='same'))

  # pooling 4
  model.add(MaxPooling2D(pool_size=(2, 2)))

  #Flattening
  model.add(Flatten())

  # camada totalmente conectada 1
  model.add(Dense(256, activation=act))

  # camada totalmente conectada 2
  model.add(Dense(256, activation=act))

  model.add(Dropout(rate=0.5))

  # camada saída
  model.add(Dense(2, activation='softmax'))

  return model

In [23]:
def update_train_test (test_size = 0.30):
  # Converte os dados com as novas imgs e labels pra um numpy array
  X = np.array(data['img'])
  Y = np.array(data['class'])

  # train test split
  entrada_train, entrada_test, saida_train, saida_test = train_test_split(X, Y, test_size=test_size)

  # normalize
  entrada_train_norm = entrada_train / 255.0
  entrada_test_norm = entrada_test / 255.0
  saida_train = to_categorical(saida_train)
  saida_test = to_categorical(saida_test)

  return entrada_train_norm, entrada_test_norm, entrada_train, entrada_test, saida_train, saida_test

In [24]:
# le os 2 diretórios
reset_data()
read_folder(mask_dir, 1)
read_folder(no_mask_dir, 0)

entrada_train_norm, entrada_test_norm, entrada_train, entrada_test, saida_train, saida_test = update_train_test()

In [25]:
model = get_model()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.fit(entrada_train_norm, saida_train, validation_data=(entrada_test_norm, saida_test), epochs=_epochs, verbose=1, batch_size=64)

(64, 64, 3)
Epoch 1/50
5/5 [==============================] - 14s 333ms/step - loss: 0.7325 - accuracy: 0.4870 - val_loss: 0.6909 - val_accuracy: 0.4470
Epoch 2/50
5/5 [==============================] - 0s 39ms/step - loss: 0.6771 - accuracy: 0.5519 - val_loss: 0.6318 - val_accuracy: 0.7576
Epoch 3/50
5/5 [==============================] - 0s 38ms/step - loss: 0.5774 - accuracy: 0.6851 - val_loss: 0.6516 - val_accuracy: 0.6591
Epoch 4/50
5/5 [==============================] - 0s 38ms/step - loss: 0.5203 - accuracy: 0.8506 - val_loss: 0.7388 - val_accuracy: 0.7424
Epoch 5/50
5/5 [==============================] - 0s 37ms/step - loss: 0.4802 - accuracy: 0.8442 - val_loss: 0.3448 - val_accuracy: 0.9091
Epoch 6/50
5/5 [==============================] - 0s 37ms/step - loss: 0.2815 - accuracy: 0.9156 - val_loss: 0.6409 - val_accuracy: 0.7955
Epoch 7/50
5/5 [==============================] - 0s 38ms/step - loss: 0.2946 - accuracy: 0.9026 - val_loss: 0.4040 - val_accuracy: 0.8485
Epoch 8/50
5/

In [26]:
model.save('/content/drive/My Drive/AI/IGTI/Bootcamp Deep Learning/Desafio Final/facemask')
tfjs.converters.save_keras_model(model, '/content/drive/My Drive/AI/IGTI/Bootcamp Deep Learning/Desafio Final/facemask-tfjs')

INFO:tensorflow:Assets written to: /content/drive/My Drive/AI/IGTI/Bootcamp Deep Learning/Desafio Final/facemask/assets


In [27]:
folder = '/content/drive/My Drive/AI/IGTI/Bootcamp Deep Learning/Desafio Final/pics'
reset_data()
read_folder(folder, 0)

x = np.array(data['img'])

In [28]:
res = model.predict(x)

In [ ]:
t = ['SIM', 'não']

c = [Fore.GREEN, Fore.RED]

i = 0

for r in res:
  cv2_imshow(x[i])
  ri = int(r[0])
  print(c[ri] + t[ri])
  print('\n\n\n')
  i = i + 1